In [9]:
import numpy as np
import pandas as pd
import seaborn as sns

In [10]:
df = pd.read_parquet("../dataset/etl/L3.correlations.parquet")
df.head()

,variable,method,r,pvalue
0,tags_bisa_nego,pointbiser,-0.014723,4.677036e-02
1,tags_cash_bertahap,pointbiser,-0.236766,8.352566e-231
2,tags_cash_keras,pointbiser,-0.146750,2.423117e-88
3,tags_dijual_cepat,pointbiser,0.016894,2.250845e-02
4,tags_komplek,pointbiser,-0.072232,1.553275e-22


In [11]:
len(df[np.abs(df["r"]) > 0.1])

31

In [12]:
len(df[df["pvalue"] < 0.05])

57

In [13]:
def formatter_pvalue(x):
  return "color: red" if x < 0.05 else None

df.style.map(formatter_pvalue, subset=["pvalue"])

,variable,method,r,pvalue
0,tags_bisa_nego,pointbiser,-0.014723,0.046770
1,tags_cash_bertahap,pointbiser,-0.236766,0.000000
2,tags_cash_keras,pointbiser,-0.146750,0.000000
3,tags_dijual_cepat,pointbiser,0.016894,0.022508
4,tags_komplek,pointbiser,-0.072232,0.000000
5,tags_kpr,pointbiser,-0.106366,0.000000
6,tags_masuk_gang,pointbiser,-0.053675,0.000000
7,tags_one_gate_system,pointbiser,-0.055082,0.000000
8,tags_pedesaan,pointbiser,-0.005735,0.438614
9,tags_perumahan,pointbiser,-0.072232,0.000000
